VSE Jonah Heyl Hungarian algorithm
This algorithm is based off other implementations found below

In [1]:
using Statistics

In [15]:
function w(v)
    w=[]
    ind=0
    for i in v
        ind=ind+1
        if i
            push!(w,ind)
        end
    end
    return w
end

w (generic function with 1 method)

In [4]:
function min_zero_row(zero_mat, mark_zero)
    min_row = [99999, -1]
    v=[]
    for i in 1:size(zero_mat)[1]
        count=0
        for j in zero_mat[i,:]
            if j
                count=count+1
            end
        end
        if count>0
            push!(v,count)
        else
            push!(v,99999)
        end
    end
    min_row=findmin(v)[2]
    zero_index = w(zero_mat[min_row,:])

    push!(mark_zero,((min_row, zero_index[1]))) 
    for i in 1:size(zero_mat)[1]
        zero_mat[min_row[1], i] = false
    end
    for j in 1:size(zero_mat)[2]
       zero_mat[j, zero_index[1]] = false
    end
    
end
    

min_zero_row (generic function with 1 method)

In [6]:
function cov(S) #converts the set into a vector
    v=[]
    for i in S
       push!(v,i)
    end
    return v
end

cov (generic function with 1 method)

In [8]:
function nope(cover_rows,row)
    for x  in cover_rows
        if (x == row)
            return false
        end
    end
    return true
end

nope (generic function with 1 method)

In [9]:
function mark_matrix(mat)
    cur_mat = mat
    v=[]
    for x in cur_mat
        if x==0
            push!(v,true)
        end
        if x!=0
            push!(v,false)
        end
    end
    zero_bool_mat= reshape(v,size(cur_mat))
    zero_bool_mat_copy = copy(zero_bool_mat)
    marked_zero = []
    while (true in zero_bool_mat_copy)
        min_zero_row(zero_bool_mat_copy, marked_zero)
    end
    marked_zero_row = []
    marked_zero_col = []
    for i in 1:size(marked_zero)[1]
        push!(marked_zero_row,(marked_zero[i,1]))
        push!(marked_zero_col,(marked_zero[i,1]))
    end
    non_marked_row=cov(setdiff(Set(1:size(cur_mat)[1]),Set(marked_zero_row)))
    marked_cols=[]
    check_switch = true
    while check_switch
        check_switch = false
        for i in 1:size(non_marked_row)[1] 
            row_array = zero_bool_mat[non_marked_row[i],:]
            for j in 1:size(row_array)[1]
                if row_array[j] == true && nope(marked_cols,j)
                    push!(marked_cols,j)
                    check_switch = true
                end
            end
        end
    end
    for row_num in marked_zero
            for col_num in marked_zero
                if nope(non_marked_row,row_num) && nope(marked_cols,col_num)==false
                    push!(non_marked_row,row_num)
                    check_switch = true
                end
            end
    end
    marked_rows = cov(setdiff(Set(1:size(mat)[1]),Set(non_marked_row)))

    return(marked_zero, marked_rows, marked_cols)
end

mark_matrix (generic function with 1 method)

In [10]:
function adjust_matrix(mat, cover_rows, cover_cols)
    cur_mat = mat
    non_zero_element = []
    for row in 1:size(cur_mat)[1]
        if nope(cover_rows,row)
            for i in 1:size(cur_mat[row])
                if nope(cover_cols,i)
                    push!(non_zero_element,cur_mat[row][i])
                end
            end
        end
    end
    min_num = findmin(non_zero_element)[1]

     
    for row in 1:size(cur_mat)[1]
        if nope(cover_rows,row)
            for i in 1:size(cur_mat[row])[1]
                if nope(cover_cols,i)
                    cur_mat[row, i] = cur_mat[row][i] - min_num
                end
            end
        end
    end
  

    for row in 1:size(cover_rows)[1]
        for col in 1:size(cover_cols)[1]
            cur_mat[cover_rows[row], cover_cols[col]] = cur_mat[cover_rows[row], cover_cols[col]] + min_num
        end
    end
    return cur_mat
end

adjust_matrix (generic function with 1 method)

In [11]:
function hungarian_algorithm(mat)
    ans_pos=0
    cur_mat = mat

    for row_num in 1:size(mat)[1]
        cur_mat[row_num] = cur_mat[row_num] .- findmin(cur_mat[row_num])[1]
    end
    
    for col_num in 1:size(mat)[2]
        cur_mat[:,col_num] = cur_mat[:,col_num] .- findmin(cur_mat[:,col_num])[1]
    end
    zero_count = 0

    while zero_count < size(mat)[1]

        ans_pos, marked_rows, marked_cols =mark_matrix(cur_mat)

        zero_count = size(marked_rows)[1] + size(marked_cols)[1]        
        if zero_count < size(mat)[1]
            cur_mat = adjust_matrix(cur_mat, marked_rows, marked_cols)
        end
    end

    return ans_pos
end
            

hungarian_algorithm (generic function with 1 method)

In [55]:
findmin([4,5,6])

(4, 1)

In [13]:
function ans_calculation(mat, pos)
    total = 0
    ans_mat = zeros(size(mat))
    for i in 1:size(pos)[1]
        total = total + (mat[pos[i][1], pos[i][2]])
        ans_mat[pos[i][1], pos[i][2]] = mat[pos[i][1], pos[i][2]]
    end
    return total, ans_mat
end

ans_calculation (generic function with 1 method)

In [51]:
A= rand(1:10,(3, 3))

3×3 Matrix{Int64}:
 4  5  2
 9  1  7
 8  5  2

In [52]:
ans_pos = hungarian_algorithm(copy(A))

3-element Vector{Any}:
 (1, 1)
 (2, 2)
 (3, 3)

In [53]:
res=ans_calculation(copy(A),ans_pos)

(7, [4.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 2.0])

In [54]:
res[2]

3×3 Matrix{Float64}:
 4.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  2.0

Also See:
https://launchpad.net/lib-bipartite-match
http://software.clapper.org/munkres/
https://en.wikipedia.org/wiki/Hungarian_algorithm
https://github.com/KevinStern/software-and-algorithms/blob/master/src/main/java/blogspot/software_and_algorithms/stern_library/optimization/HungarianAlgorithm.java
https://python.plainenglish.io/hungarian-algorithm-introduction-python-implementation-93e7c0890e15
https://github.com/maandree/hungarian-algorithm-n3/blob/master/hungarian.c